# Notebook for testing MPS and MPU devices on Apple Silicon Macs

In [1]:
import platform
import sys
import pandas as pd
import numpy as np
import sklearn as sk
import scipy as sp
import tensorflow as tf
import keras
import torch

print(f"Python {sys.version}")
print(f"Python Platform: {platform.platform ()}")
print()
print(f"Pandas {pd.__version__}")
print(f"NumPy {np.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
print()
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

# Check if GPU for Tensorflow/Keras is available
gpu = len(tf.config.list_physical_devices("GPU")) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
print()
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Python 3.11.10 | packaged by conda-forge | (main, Sep 22 2024, 14:11:13) [Clang 17.0.6 ]
Python Platform: macOS-14.6.1-arm64-arm-64bit

Pandas 2.2.3
NumPy 1.26.4
Scikit-Learn 1.5.2
SciPy 1.14.1

Tensor Flow Version: 2.14.0
Keras Version: 2.14.0
GPU is available

PyTorch version: 2.4.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [2]:
# Show the configuration of NumPy
np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4377784592
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -ftree-vectorize, -fPIC, -fstack-protector-strong, -O2, -pipe, -isystem,
      /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include, -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/numpy_1707225640867/work=/usr/local/src/conda/numpy-1.26.4,
      -fdebug-prefix-map=/opt/homebrew/Caskroom/miniforge/base/envs/ML_Zo

In [3]:
# Show the configuration of SciPy
sp.__config__.show()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: lapack
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  pybind11:
    detection method: pkgconfig
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    name: pybind11
    version: 2.13.4
Compilers:
  c: